<a href="https://colab.research.google.com/github/Edisuism/Machine_Learning/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2


In this page, I will develop a machine learning algorithm from scratch and then test it on a sample dataset

Steps involved:
- Find sample dataset
- Choose learner algorithm and note down input/output
- Preprocess sample dataset to input requirements and split into test/training sets 
- code learner algorithm **(only this needs to be coded, preprocessing can use from libaries like SKlearn)**
- train algorithm and then apply to test set

# Introduction

new text


# Exploration

# Methodology

Import data


In [0]:
import numpy as np
import pandas as pd

import random

url = 'https://raw.githubusercontent.com/Edisuism/Machine_Learning/master/play_tennis.csv'
df = pd.read_csv(url)
print (len(df))
df.head()

14


,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


Split into training and testing sets

In [0]:
def train_test_split(df, test_size):
  test_size = round(test_size * len(df))
  indices = df.index.tolist()
  test_indices = random.sample(population = indices, k = test_size)
  test_df = df.loc[test_indices]
  train_df = df.drop(test_indices)
  
  return train_df, test_df



In [0]:
train_df, test_df = train_test_split(df, test_size = 0.3)

In [0]:
print (len(train_df))
train_df.head()

10


,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
6,D7,Overcast,Cool,Normal,Strong,Yes


In [0]:
print (len(test_df))
test_df.head()

4


,day,outlook,temp,humidity,wind,play
13,D14,Rain,Mild,High,Strong,No
11,D12,Overcast,Mild,High,Strong,Yes
5,D6,Rain,Cool,Normal,Strong,No
3,D4,Rain,Mild,High,Weak,Yes


Notes:
Gini index = how much uncertainty there is in a node
Information gain = how much uncertainty is removed in a node

# Evaluation

# Conclusion

# Ethical